In [ ]:
import time
import os
import pandas as pd
import h5py
import numpy as np
from multiprocessing import Pool
from multiprocessing.pool import ThreadPool
from functools import partial, reduce
from collections import deque
from IPython.core.debugger import set_trace

# labelBaseMap = {
#     0: "A",
#     1: "C",
#     2: "G",
#     3: "T"
# }

possible_filenames = ["/mnt/nvme/taiyaki_aligned/mapped_umi16to9.hdf5",
                      "/Users/felix/MsC/DNA/mapped_umi16to9.hdf5"]

RNN_LEN = 200

In [ ]:
def normalise(dac):
    dmin = min(dac)
    dmax = max(dac)
    return [(d-dmin)/(dmax-dmin) for d in dac]

def ohe(v):
    tr = [0,0,0,0]
    tr[v] = 1
    return tr

def change_shape(dac):
    return [[x] for x in dac]

In [ ]:
for filename in possible_filenames:
    if not os.path.isfile(filename):
        pass
    with h5py.File(filename, 'r') as h5file:
        readIDs = list(h5file['Reads'].keys())
        print(f"{len(readIDs)} reads, keys: {list(h5file['Reads'][readIDs[0]].keys())}")
    break

In [ ]:
def processRead(readID, filename, train_validate_split=0.8, min_labels=5, one_hot_encode=False):
    train_X = []
    train_y = []
    test_X  = []
    test_y  = []
    with h5py.File(filename, 'r') as h5file:
        DAC = list(normalise(h5file['Reads'][readID]['Dacs'][()]))
        RTS = deque(list(h5file['Reads'][readID]['Ref_to_signal'][()]))
        if ohe:
            REF = deque(h5file['Reads'][readID]['Reference'][()])
        else:
            REF = deque(h5file['Reads'][readID]['Reference'][()])
        
    # just get the number, (1-tvs) so that it can be compared to how many items are left    
    train_validate_split = round(len(REF)*(1-train_validate_split))
    
    # -5 so that the first WHILE iteration afterwards cancels it out
    curdacs  = deque( [[x] for x in DAC[RTS[0]:RTS[0]+RNN_LEN-5]], RNN_LEN ) # deque keeping `RNN_LEN` DACs
    curdacts = RTS[0]+RNN_LEN-5 # the current HEAD
    labels  = deque([]) # to hold the label for the sequence
    labelts = deque([]) # to hold the timestamps for the labels

    while RTS[0] < curdacts: # add the first RNN_LEN worth of labels to initialise
        labels.append(REF.popleft())
        labelts.append(RTS.popleft())    
    
    
    while curdacts+5 < RTS[-1]-RNN_LEN:
        curdacs.extend([[x] for x in DAC[curdacts:curdacts+5]])
        curdacts += 5
        
        # add labels if new ones appeared
        while RTS[0] < curdacts:
            labels.append(REF.popleft())
            labelts.append(RTS.popleft())    
        
        # pop from labels if we passed them
        # sometimes the strand gets stuck and the deques drop to 0
        while len(labelts) > 0 and labelts[0] < curdacts - RNN_LEN:
            labels.popleft()
            labelts.popleft()

        # only add if more than 5 labels
        if len(labels) > min_labels:
            # add to train if more than tvs remain
            if len(RTS) > train_validate_split:
                train_X.append(list(curdacs))
                train_y.append(list(labels))
            else:
                test_X.append(list(curdacs))
                test_y.append(list(labels))
                
    return train_X, train_y, test_X, test_y

# pp = partial(processRead, filename=filename, one_hot_encode=True)
# tr_X, tr_y, te_X, te_y = pp(readIDs[1])

In [ ]:
pool = Pool(16)
results_prim = pool.map(partial(processRead, filename=filename, one_hot_encode=True), readIDs[:16])
pool.close()
pool.join()

print("Pool done")

train_X = []
train_y = []
test_X  = []
test_y  = []
for thread in results_prim:
    train_X.extend(thread[0])
    train_y.extend(np.array(thread[1]))
    test_X.extend(thread[2])
    test_y.extend(thread[3])

train_X = np.array(train_X)
train_y = np.array(train_y)
test_X  = np.array(test_X)
test_y  = np.array(test_y)


print(train_X.shape)
print(train_y.shape)
print(test_X.shape)
print(test_y.shape)

In [ ]:
# np.save("train_X", train_X)
# np.save("train_y", train_y)
# np.save("test_X", test_X)
# np.save("test_y", test_y)

In [ ]:
# train_X = np.load("train_X.npy")
# train_y = np.load("train_y.npy", allow_pickle=True)
# test_X  = np.load("test_X.npy")
# test_y  = np.load("test_y.npy", allow_pickle=True)


# print(train_X.shape)
# print(train_y.shape)
# print(test_X.shape)
# print(test_y.shape)

# HERE COME DAT ML

In [ ]:
import tensorflow as tf
import tensorflow.keras.backend as kb
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Activation, Add, Lambda
from tensorflow.keras.layers import Dense, MaxPooling1D, Conv1D, LSTM
from tensorflow.keras.backend import ctc_batch_cost
import numpy as np

# device_hasgpu = tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
# if device_hasgpu:
#     gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
#     sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

In [ ]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return kb.ctc_batch_cost(labels, y_pred, input_length, label_length)

In [ ]:
input_data = Input(name="input", shape=(200,1), dtype="float32")
inner = Conv1D(32, 3,
          padding="valid",
          activation="relu", 
          input_shape=(200,1),
          name="conv1d_1")(input_data)
inner = MaxPooling1D(pool_size=5, name="maxpool_1")(inner)
lstm_1a = LSTM(32,return_sequences=True, name="lstm_1a")(inner)
lstm_1b = LSTM(32, return_sequences=True, go_backwards=True, name="lstm_1b")(inner)
lstm_1_merged = Add()([lstm_1a, lstm_1b])

inner = Dense(5, name="dense_1")(lstm_1_merged)

y_pred = Activation("softmax", name="softmax")(inner)

# Model(inputs=input_data, outputs=y_pred).summary()

labels = Input(name='the_labels', shape=[5], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out, name="my_model")
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer='adam')

model.summary()

In [ ]:
inputs = {
    'the_input': train_X,
    'the_labels': train_y,
    'input_length': np.array([[39-2]]*len(train_X)),
    'label_length': np.array([[len(y)] for y in train_y])
}

outputs = {'ctc': np.zeros([len(train_X)])} 

In [ ]:
model.fit(inputs)